### Pre-process position data
J. Carpenter 2020

In [6]:
# libraries
import scipy.io as sio
import numpy as np

In [4]:
# load dataset for animal 24116
fileLoc = "D:\\Data\\Dataset\\24116_2.mat" # insert your file location here
matfile = sio.loadmat(fileLoc) # read in .mat file as dictionary

# parse matfile
accel = matfile["accel"][0]
behaviour = matfile["behaviour"][0]
hd = matfile["hd"][0]
pos = matfile["pos_cm"][0]
rawEEG = matfile["rawEEG"][0]
sessInfo = matfile["sessInfo"][0]
speed = matfile["speed"][0]
SpikeTimes = matfile["SpikeTimes"][0]
SpikeTrain = matfile["SpikeTrain"][0]
trialType = matfile["trialType"][0]
UniqueID = matfile["UniqueID"][0]

# Manually define home-well locations for each session (I'll incorporate this into the dataset later)
hwLoc = [37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
         37, 37, 37, 36, 37, 36, 36, 36, 36, 37, 37, 37, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37]

C:\Users\17145\Anaconda3\lib\site-packages\scipy\io\matlab\mio.py:224: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [5]:
# preprocess the position data- remove outliers
def remove_outliers_LED(pos, matfile, sessNum=0):
    # remove outliers and convert to cm
    conversionFactor = 2.6
    minX = float(matfile["sessInfo"][0][sessNum][0]["window_min_x"][0][0][0][0])
    minY = float(matfile["sessInfo"][0][sessNum][0]["window_min_y"][0][0][0][0])

    pos_ = np.zeros([len(pos[sessNum][:,0]), 5])
    pos_[:,0] = pos[sessNum][:,0]
    pos_[:,1] = (pos[sessNum][:,1]- minX)/conversionFactor
    pos_[:,2] = (pos[sessNum][:,2]- minY)/conversionFactor
    pos_[:,3] = (pos[sessNum][:,3]- minX)/conversionFactor
    pos_[:,4] = (pos[sessNum][:,4]- minY)/conversionFactor

    # remove outliers
    x = pos_[:,1]; y = pos_[:,2]
    x2 = pos_[:,3]; y2 = pos_[:,4]

    d = np.zeros([len(x)])
    x_prime = np.zeros([len(x)]); y_prime = np.zeros([len(x)])
    x2_prime = np.zeros([len(x)]); y2_prime = np.zeros([len(x)])
    corrPos =  np.zeros([len(x), 5]);

    for idx,val in enumerate(x):
        if idx < len(x)-1:
            d[idx] = np.sqrt((x[idx+1] - x[idx])**2 + (y[idx+1] - y[idx])**2)
            if d[idx] < 2:
                x_prime[idx:idx+1] = x[idx:idx+1]; y_prime[idx:idx+1] = y[idx:idx+1];
                x2_prime[idx:idx+1] = x2[idx:idx+1]; y2_prime[idx:idx+1] = y2[idx:idx+1];

            else:
                x_prime[idx:idx+1] = np.nan; y_prime[idx:idx+1] = np.nan;
                x2_prime[idx:idx+1] = np.nan; y2_prime[idx:idx+1] = np.nan;

    corrPos[:,0]=pos_[:,0]; 
    corrPos[:,1]=x_prime + np.absolute(np.nanmin(x_prime)); corrPos[:,2]=y_prime + np.absolute(np.nanmin(y_prime));
    corrPos[:,3]=x2_prime + np.absolute(np.nanmin(x2_prime)); corrPos[:,4]=y2_prime + np.absolute(np.nanmin(y2_prime))

    return corrPos

# run for all sessions
pos_outliers_removed = {} # create dictionary
for ii in range(0,len(pos)):
    corrPos = remove_outliers_LED(pos, matfile, sessNum=ii)
    pos_outliers_removed[ii] = corrPos